# 01. データ準備 - S3画像収集 & Ground Truthアノテーションジョブ作成

## 概要
1. S3に蓄積されたメーター画像を一覧取得（hygrometer-webcam, hygrometer-scam のみ）
2. Ground Truth用のマニフェストファイルを作成
3. カスタムUIテンプレート（crowd-keypoint）を作成
4. Ground Truthキーポイントアノテーションジョブを起動

## リージョン
- ap-northeast-1（東京）

## 1. セットアップ

In [ ]:
import boto3
import json
import os
from datetime import datetime
from pathlib import Path

# AWS クライアント（東京リージョン）
REGION = 'ap-northeast-1'
s3 = boto3.client('s3', region_name=REGION)
sagemaker_client = boto3.client('sagemaker', region_name=REGION)

# 設定
SOURCE_BUCKET = 'facteye-images-20251114'  # 既存の画像バケット
BASE_PREFIX = 'ml-datasets/v2'

# 対象メーター種別（湿度計のみ）
TARGET_PREFIXES = [
    f'{BASE_PREFIX}/hygrometer-webcam',
    f'{BASE_PREFIX}/hygrometer-scam',
]

# SageMaker デフォルトバケット取得
import sagemaker
sess = sagemaker.Session()
SAGEMAKER_BUCKET = sess.default_bucket()
ROLE = sagemaker.get_execution_role()

print(f'Region: {REGION}')
print(f'Source Bucket: {SOURCE_BUCKET}')
print(f'SageMaker Bucket: {SAGEMAKER_BUCKET}')
print(f'Target Prefixes: {TARGET_PREFIXES}')
print(f'Role: {ROLE}')

## 2. S3画像一覧取得 & フィルタリング

対象プレフィックス（hygrometer-webcam, hygrometer-scam）から画像を収集する。

In [ ]:
def list_s3_images(bucket, prefix, extensions=('.jpg', '.jpeg', '.png', '.bmp')):
    """S3から指定プレフィックス配下の画像ファイルを一覧取得"""
    images = []
    paginator = s3.get_paginator('list_objects_v2')
    
    for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
        for obj in page.get('Contents', []):
            key = obj['Key']
            if any(key.lower().endswith(ext) for ext in extensions):
                images.append({
                    'key': key,
                    'meter_type': prefix,
                    'size': obj['Size'],
                    'last_modified': obj['LastModified'],
                })
    return images

# 対象プレフィックスから画像を収集
filtered_images = {}
for prefix in TARGET_PREFIXES:
    imgs = list_s3_images(SOURCE_BUCKET, prefix)
    filtered_images[prefix] = imgs
    print(f'{prefix}: {len(imgs)} 枚')

total = sum(len(v) for v in filtered_images.values())
print(f'\n合計: {total} 枚')

## 3. Ground Truth マニフェストファイル作成

Ground Truthジョブに必要な入力マニフェスト（JSONL形式）を生成する。

In [ ]:
def create_manifest(images, bucket, output_path):
    """Ground Truth用のマニフェストファイルを作成
    
    各行のJSON形式:
    {"source-ref": "s3://bucket/key", "meter_type": "..."}
    """
    manifest_lines = []
    
    for img in images:
        entry = {
            'source-ref': f's3://{bucket}/{img["key"]}',
            'meter_type': img.get('meter_type', 'unknown'),
        }
        manifest_lines.append(json.dumps(entry))
    
    manifest_content = '\n'.join(manifest_lines)
    
    # S3にアップロード
    s3.put_object(
        Bucket=SAGEMAKER_BUCKET,
        Key=output_path,
        Body=manifest_content.encode('utf-8'),
        ContentType='application/jsonl'
    )
    
    print(f'マニフェスト作成完了: s3://{SAGEMAKER_BUCKET}/{output_path}')
    print(f'  画像数: {len(manifest_lines)}')
    return f's3://{SAGEMAKER_BUCKET}/{output_path}'

# 全対象画像をまとめたマニフェストを作成
all_target_images = []
for meter_type, imgs in filtered_images.items():
    all_target_images.extend(imgs)

# タイムスタンプはジョブ名に使用（マニフェストは固定キーで上書き）
timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')
manifest_s3_uri = create_manifest(
    all_target_images,
    SOURCE_BUCKET,
    'ground-truth/manifests/input.manifest'
)

print(f'\n合計画像数: {len(all_target_images)}')

## 4. カスタムUIテンプレート（crowd-keypoint）

アナログメーターのキーポイントアノテーション用UIテンプレートを作成する。

### キーポイント定義
| キーポイント | 説明 |
|-------------|------|
| `needle_tip` | 針の先端 |
| `needle_center` | 針の回転中心（軸） |
| `scale_min` | スケール最小値の位置 |
| `scale_max` | スケール最大値の位置 |

In [ ]:
UI_TEMPLATE = """
<script src="https://assets.crowd.aws/crowd-html-elements.js"></script>

<crowd-form>
  <crowd-keypoint
    src="{{ task.input.source-ref | grant_read_access }}"
    labels="['needle_tip', 'needle_center', 'scale_min', 'scale_max']"
    header="メーターのキーポイントをアノテーションしてください"
    name="keypoints"
  >
    <full-instructions header="アノテーション手順">
      <h2>キーポイントの定義</h2>
      <p>各メーター画像に対して、以下の4つのキーポイントを正確にマークしてください。</p>
      <table border="1" cellpadding="5">
        <tr><th>キーポイント</th><th>説明</th><th>注意事項</th></tr>
        <tr>
          <td><strong>needle_tip</strong></td>
          <td>針の先端</td>
          <td>針が指し示している最も外側の端をマーク</td>
        </tr>
        <tr>
          <td><strong>needle_center</strong></td>
          <td>針の回転中心（軸）</td>
          <td>針が回転する中心点をマーク</td>
        </tr>
        <tr>
          <td><strong>scale_min</strong></td>
          <td>スケール最小値の位置</td>
          <td>目盛りの最小値（0やMINなど）の位置をマーク</td>
        </tr>
        <tr>
          <td><strong>scale_max</strong></td>
          <td>スケール最大値の位置</td>
          <td>目盛りの最大値の位置をマーク</td>
        </tr>
      </table>

      <h2>画像がぼやけている場合</h2>
      <p>画像がぼやけていて判別困難な場合でも、最善の推測でキーポイントをマークしてください。</p>
      <p>どうしても判別できない場合は、全てのキーポイントを画像の中心に配置してください（後処理で除外します）。</p>

      <h2>メーター種別: {{ task.input.meter_type }}</h2>
    </full-instructions>

    <short-instructions>
      <p>4つのキーポイントをマークしてください:</p>
      <ol>
        <li><strong>needle_tip</strong>: 針の先端</li>
        <li><strong>needle_center</strong>: 針の回転中心</li>
        <li><strong>scale_min</strong>: スケール最小値</li>
        <li><strong>scale_max</strong>: スケール最大値</li>
      </ol>
      <p>メーター種別: {{ task.input.meter_type }}</p>
    </short-instructions>
  </crowd-keypoint>
</crowd-form>
"""

# UIテンプレートをS3にアップロード
template_s3_key = 'ground-truth/ui-template/keypoint-template.liquid.html'
s3.put_object(
    Bucket=SAGEMAKER_BUCKET,
    Key=template_s3_key,
    Body=UI_TEMPLATE.encode('utf-8'),
    ContentType='text/html'
)

print(f'UIテンプレートアップロード完了: s3://{SAGEMAKER_BUCKET}/{template_s3_key}')

## 5. プライベートワークフォース確認

既存のプライベートワークフォースを確認する。存在しない場合はSageMakerコンソールから作成する。

In [ ]:
# プライベートワークフォースの確認
try:
    workforces = sagemaker_client.list_workforces()
    if workforces['Workforces']:
        workforce = workforces['Workforces'][0]
        workforce_arn = workforce['WorkforceArn']
        print(f'ワークフォース検出: {workforce["WorkforceName"]}')
        print(f'ARN: {workforce_arn}')
    else:
        print('ワークフォースが見つかりません。')
        print('SageMakerコンソール > Ground Truth > Labeling workforces から作成してください。')
except Exception as e:
    print(f'ワークフォース確認エラー: {e}')
    print('SageMakerコンソールからプライベートワークフォースを作成してください。')

In [ ]:
# プライベートワークチームの確認
try:
    workteams = sagemaker_client.list_workteams()
    if workteams['Workteams']:
        for team in workteams['Workteams']:
            print(f'ワークチーム: {team["WorkteamName"]}')
            print(f'  ARN: {team["WorkteamArn"]}')
    else:
        print('ワークチームが見つかりません。')
        print('SageMakerコンソールからワークチームを作成してください。')
except Exception as e:
    print(f'ワークチーム確認エラー: {e}')

## 5.5 カスタムLambda関数のデプロイ

Ground Truth ジョブで使用する **Pre-annotation / Annotation Consolidation (ACS)** Lambda をデプロイする。

| Lambda | 役割 | 処理内容 |
|--------|------|----------|
| Pre-annotation | マニフェスト各行 → UIテンプレート用に整形 | ほぼパススルー |
| ACS | ワーカーの回答を集約 → output manifest | ワーカー1人なのでそのまま採用 |

### デプロイされるリソース
1. **IAMロール** (`facteye-gt-lambda-role`) — Lambda実行権限 + S3読み取り
2. **Pre Lambda** (`facteye-gt-pre-keypoint`) — 前処理
3. **ACS Lambda** (`facteye-gt-acs-keypoint`) — 後処理（集約）
4. **リソースポリシー** — SageMaker からの呼び出し許可

In [ ]:
# ---- Pre-annotation Lambda ----
# マニフェストの各行をそのまま taskInput として返す（パススルー）
PRE_LAMBDA_CODE = """
import json

def handler(event, context):
    return {
        "taskInput": event["dataObject"],
        "isHumanAnnotationRequired": "true"
    }
"""

# ---- Annotation Consolidation (ACS) Lambda ----
# NumberOfHumanWorkersPerDataObject=1 のため、唯一のワーカー回答をそのまま採用
ACS_LAMBDA_CODE = """
import json
import boto3
from urllib.parse import urlparse
from datetime import datetime, timezone

s3 = boto3.client("s3")

def handler(event, context):
    # S3 からワーカーのアノテーション結果を取得
    parsed = urlparse(event["payload"]["s3Uri"])
    obj = s3.get_object(Bucket=parsed.netloc, Key=parsed.path.lstrip("/"))
    annotations = json.loads(obj["Body"].read().decode("utf-8"))

    label_attr = event["labelAttributeName"]
    results = []

    for item in annotations:
        worker_data = json.loads(
            item["annotations"][0]["annotationData"]["content"]
        )
        results.append({
            "datasetObjectId": item["datasetObjectId"],
            "consolidatedAnnotation": {
                "content": {
                    label_attr: worker_data,
                    f"{label_attr}-metadata": {
                        "type": "groundtruth/custom",
                        "human-annotated": "yes",
                        "creation-date": datetime.now(timezone.utc).isoformat(),
                    },
                }
            },
        })

    return results
"""

print("=== Pre-annotation Lambda ===")
print(PRE_LAMBDA_CODE)
print("\n=== ACS Lambda ===")
print(ACS_LAMBDA_CODE)

In [ ]:
import time
import zipfile
import io

lambda_client = boto3.client('lambda', region_name=REGION)
iam_client = boto3.client('iam')
sts_client = boto3.client('sts')

ACCOUNT_ID = sts_client.get_caller_identity()['Account']
LAMBDA_ROLE_NAME = 'facteye-gt-lambda-role'
PRE_FUNCTION_NAME = 'facteye-gt-pre-keypoint'
ACS_FUNCTION_NAME = 'facteye-gt-acs-keypoint'

# ---- 1. IAM ロール作成 ----
trust_policy = {
    "Version": "2012-10-17",
    "Statement": [{
        "Effect": "Allow",
        "Principal": {"Service": "lambda.amazonaws.com"},
        "Action": "sts:AssumeRole"
    }]
}

try:
    role_resp = iam_client.create_role(
        RoleName=LAMBDA_ROLE_NAME,
        AssumeRolePolicyDocument=json.dumps(trust_policy),
        Description='IAM role for Ground Truth custom Lambda functions'
    )
    lambda_role_arn = role_resp['Role']['Arn']
    print(f'IAMロール作成: {lambda_role_arn}')

    # 基本実行ポリシー（CloudWatch Logs）
    iam_client.attach_role_policy(
        RoleName=LAMBDA_ROLE_NAME,
        PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
    )

    # S3読み取りポリシー（ACS Lambda がアノテーション結果を読むため）
    iam_client.put_role_policy(
        RoleName=LAMBDA_ROLE_NAME,
        PolicyName='S3ReadForConsolidation',
        PolicyDocument=json.dumps({
            "Version": "2012-10-17",
            "Statement": [{
                "Effect": "Allow",
                "Action": ["s3:GetObject"],
                "Resource": f"arn:aws:s3:::{SAGEMAKER_BUCKET}/*"
            }]
        })
    )

    # IAMロールの伝播待ち（Lambda作成時に Role が見つからないエラーを防ぐ）
    print('IAMロール伝播を待機中（10秒）...')
    time.sleep(10)

except iam_client.exceptions.EntityAlreadyExistsException:
    lambda_role_arn = f'arn:aws:iam::{ACCOUNT_ID}:role/{LAMBDA_ROLE_NAME}'
    print(f'IAMロール既存: {lambda_role_arn}')

print(f'Lambda Role ARN: {lambda_role_arn}')

# ---- 2. Lambda 関数デプロイ ----
def create_lambda_zip(code_str):
    """Lambda関数コードをZIPバイト列に変換"""
    buf = io.BytesIO()
    with zipfile.ZipFile(buf, 'w', zipfile.ZIP_DEFLATED) as zf:
        zf.writestr('lambda_function.py', code_str)
    buf.seek(0)
    return buf.read()

def deploy_lambda(function_name, code_str, description):
    """Lambda関数をデプロイ（既存なら更新）"""
    zip_bytes = create_lambda_zip(code_str)

    try:
        resp = lambda_client.create_function(
            FunctionName=function_name,
            Runtime='python3.12',
            Role=lambda_role_arn,
            Handler='lambda_function.handler',
            Code={'ZipFile': zip_bytes},
            Description=description,
            Timeout=60,
            MemorySize=128
        )
        print(f'Lambda 作成: {function_name}')
        waiter = lambda_client.get_waiter('function_active_v2')
        waiter.wait(FunctionName=function_name)

    except lambda_client.exceptions.ResourceConflictException:
        resp = lambda_client.update_function_code(
            FunctionName=function_name,
            ZipFile=zip_bytes
        )
        print(f'Lambda 更新: {function_name}')
        waiter = lambda_client.get_waiter('function_updated_v2')
        waiter.wait(FunctionName=function_name)

    return resp['FunctionArn']

pre_lambda_arn = deploy_lambda(
    PRE_FUNCTION_NAME, PRE_LAMBDA_CODE,
    'Ground Truth Pre-annotation Lambda for keypoint passthrough'
)
print(f'  ARN: {pre_lambda_arn}')

acs_lambda_arn = deploy_lambda(
    ACS_FUNCTION_NAME, ACS_LAMBDA_CODE,
    'Ground Truth Annotation Consolidation Lambda for keypoint'
)
print(f'  ARN: {acs_lambda_arn}')

# ---- 3. SageMaker からの呼び出し許可を付与 ----
for func_name in [PRE_FUNCTION_NAME, ACS_FUNCTION_NAME]:
    try:
        lambda_client.add_permission(
            FunctionName=func_name,
            StatementId='SageMakerInvoke',
            Action='lambda:InvokeFunction',
            Principal='sagemaker.amazonaws.com'
        )
        print(f'SageMaker 呼び出し許可付与: {func_name}')
    except lambda_client.exceptions.ResourceConflictException:
        print(f'SageMaker 呼び出し許可（既存）: {func_name}')

print(f'\n=== デプロイ完了 ===')
print(f'Pre Lambda ARN:  {pre_lambda_arn}')
print(f'ACS Lambda ARN:  {acs_lambda_arn}')

## 6. Ground Truth アノテーションジョブ作成

**前提条件:**
- プライベートワークフォース & ワークチームが作成済みであること
- 上記セルで `WORKTEAM_ARN` を確認済みであること

In [ ]:
# === ジョブ設定 ===
# ワークチームARNを設定（上のセルで確認した値を入力）
WORKTEAM_ARN = 'arn:aws:sagemaker:ap-northeast-1:886557786576:workteam/private-crowd/meter-reading_team'

# 再実行のたびに新しいジョブ名を生成
timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')
JOB_NAME = f'facteye-meter-keypoint-{timestamp}'
OUTPUT_S3_URI = f's3://{SAGEMAKER_BUCKET}/ground-truth/output/'

print(f'ジョブ名: {JOB_NAME}')
print(f'入力マニフェスト: {manifest_s3_uri}')
print(f'出力先: {OUTPUT_S3_URI}')
print(f'ワークチーム: {WORKTEAM_ARN}')

In [ ]:
# ラベリングジョブの作成（カスタムLambda ARN を使用）
labeling_job_config = {
    'LabelingJobName': JOB_NAME,
    'LabelAttributeName': 'keypoints',
    'InputConfig': {
        'DataSource': {
            'S3DataSource': {
                'ManifestS3Uri': manifest_s3_uri
            }
        }
    },
    'OutputConfig': {
        'S3OutputPath': OUTPUT_S3_URI
    },
    'RoleArn': ROLE,
    'HumanTaskConfig': {
        'WorkteamArn': WORKTEAM_ARN,
        'UiConfig': {
            'UiTemplateS3Uri': f's3://{SAGEMAKER_BUCKET}/{template_s3_key}'
        },
        'PreHumanTaskLambdaArn': pre_lambda_arn,
        'TaskTitle': 'FactEye メーターキーポイントアノテーション',
        'TaskDescription': 'アナログメーター画像にキーポイント（針先端・中心・最小値・最大値）をマークしてください',
        'NumberOfHumanWorkersPerDataObject': 1,
        'TaskTimeLimitInSeconds': 300,
        'AnnotationConsolidationConfig': {
            'AnnotationConsolidationLambdaArn': acs_lambda_arn
        }
    }
}

print('ジョブ設定:')
print(json.dumps(labeling_job_config, indent=2, default=str))

In [ ]:
# ジョブ起動（実行前に設定を確認してください）
# response = sagemaker_client.create_labeling_job(**labeling_job_config)
# print(f'ラベリングジョブ起動完了: {JOB_NAME}')
# print(f'ARN: {response["LabelingJobArn"]}')

print('=== ジョブ起動はコメントアウトされています ===')
print('設定を確認後、上のコメントを解除して実行してください。')
print(f'  - WORKTEAM_ARN が正しいか確認')
print(f'  - マニフェストの画像数: {len(all_target_images)}')

## 7. ジョブの進捗確認

In [ ]:
def check_labeling_job_status(job_name):
    """ラベリングジョブの進捗を確認"""
    response = sagemaker_client.describe_labeling_job(LabelingJobName=job_name)
    
    status = response['LabelingJobStatus']
    counters = response.get('LabelCounters', {})
    
    print(f'ジョブ名: {job_name}')
    print(f'ステータス: {status}')
    print(f'総データ数: {counters.get("TotalLabeled", 0) + counters.get("Unlabeled", 0)}')
    print(f'ラベル済み: {counters.get("TotalLabeled", 0)}')
    print(f'未ラベル: {counters.get("Unlabeled", 0)}')
    print(f'失敗: {counters.get("FailedNonRetryableError", 0)}')
    
    if status == 'Completed':
        output_uri = response['OutputConfig']['S3OutputPath']
        print(f'\n出力先: {output_uri}')
        print(f'出力マニフェスト: {response.get("LabelingJobOutput", {}).get("OutputDatasetS3Uri", "N/A")}')
    
    return status

# ジョブ名を指定して確認
# check_labeling_job_status(JOB_NAME)

## 次のステップ

Ground Truthジョブが完了したら、**02_annotation_to_yolo.ipynb** でアノテーション結果をYOLOv8-pose形式に変換します。